In [ ]:
# 用rosetta对初始结构建模
import os
import subprocess

def pdb_modify_name(pdb_pwd, out_pdb_pwd):
    with open(pdb_pwd, "r") as ifile:
        lines = ifile.readlines()
        out_lines = []
        for line in lines:
            if not line.startswith("ATOM"):
                continue
            resid = line[22:26]
            res_atom = line[12:16].strip()
            atom_type = line[77]
            if int(resid) == 1 and res_atom in ["P", "OP1", "OP2"]:
                continue
            elif atom_type == "H":
                continue
            out_lines.append(line)
    with open(out_pdb_pwd, "w") as ifile:
        for line in out_lines:
            ifile.write(line)

def sbatch_farfar2(work_dir, seq, rna_ss, out_pdb):
    start_pwd = os.getcwd()
    os.makedirs(work_dir, exist_ok=True)
    os.chdir(work_dir)
    with open(f"farfar2.slurm", "w") as ifile:
        ifile.write(f"""#!/bin/bash
#SBATCH --job-name=farfar2
#SBATCH --output=farfar2.out
#SBATCH --error=farfar2.err
#SBATCH --nodes=1
#SBATCH --cpus-per-task=8
#SBATCH --gres=gpu:1
#SBATCH --time=240:00:00
#SBATCH --partition=4090

source /mnt/beegfs/opt/module/tools/modules/init/bash
module load rosetta/3.14-mpi

rna_denovo.cxx11threadmpiserialization.linuxgccrelease -sequence "{seq.lower()}" -secstruct "{rna_ss}" -nstruct 1 -out:file:silent test.out -minimize_rna true -overwrite true
rna_minimize.cxx11threadmpiserialization.linuxgccrelease -in:file:silent test.out -out:pdb true
cp S_000001_minimize.pdb {out_pdb}
"""
)
    subprocess.run(f"sbatch farfar2.slurm", shell=True)
    os.chdir(start_pwd)


input_txt = "/gene/home/ldw/Desktop/RNA_ensemble/input/dna_repair.txt"
save_dir = "/gene/home/ldw/Desktop/RNA_ensemble/input/dna_repair"
rna_infos = {}
with open(input_txt, "r") as ifile:
    for line in ifile.readlines():
        words = line.split()
        rna_infos[words[0]] = {"seq": words[1], "rna_ss": words[2]}

for name in rna_infos:
    seq = rna_infos[name]['seq']
    rna_ss = rna_infos[name]['rna_ss']
    if len(seq) != len(rna_ss):
        print(name, len(seq), len(rna_ss), seq, rna_ss)
    work_dir = os.path.join(save_dir, name)
    out_pdb = os.path.join(save_dir, f"{name}.pdb")
    sbatch_farfar2(work_dir, seq, rna_ss, out_pdb)

Submitted batch job 210847


In [3]:
# 对rosetta输出的结构进行清洗
for name in rna_infos:
    out_pdb = os.path.join(save_dir, f"{name}.pdb")
    pdb_modify_name(out_pdb, out_pdb)

In [1]:
# 基于rosetta产生的初始结构进行模拟
from md import construct_top, run_md
import os


traj_ns = 500  # 500 ns
num_of_replica = 3  # 平行轨迹数
box_d = 12  # 水盒子大小
k_m = 0.12  # 0.12 M K+
mg_m = 0.02  # 0.02 M Mg2+
temp = 400  # 400 K
ff = "ljbb"  # RNA force field
wat_ff = "opc3"  # water force field
input_txt = "/gene/home/ldw/Desktop/RNA_ensemble/input/input-primirna.fasta"
save_dir = "/gene/home/ldw/Desktop/RNA_ensemble/input"  # 初始pdb结构路径
md_pwd = "/gene/home/ldw/Desktop/RNA_ensemble/md"  # md路径
os.makedirs(md_pwd, exist_ok=True)

rna_infos = {}
with open(input_txt, "r") as ifile:
    for line in ifile.readlines():
        words = line.split()
        rna_infos[words[0]] = {"seq": words[1], "rna_ss": words[2]}

for rna_name in list(rna_infos.keys())[:]:
    pdb_pwd = os.path.join(save_dir, f"{rna_name}.pdb")
    rna_ss = rna_infos[rna_name]["rna_ss"]
    seq = rna_infos[rna_name]["seq"]
    work_pwd = os.path.join(md_pwd, f"{rna_name}_{ff}_{wat_ff}_{temp}K_{traj_ns}ns_{k_m}k_{mg_m}mg")
    top = construct_top(pdb_pwd, seq, rna_ss, work_pwd, mg_m, k_m, box_d, ff, wat_ff)
    top = "system.hmr.top"
    for replica in range(num_of_replica):
        run_md(work_pwd, top, replica, temp=temp, ns=traj_ns)

/gene/home/ldw/Desktop/RNA_ensemble/md/pri-miR159a_ljbb_opc3_400K_500ns_0.12k_0.02mg
Water molecules: 9840
-I: Adding /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/prep to search path.
-I: Adding /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/lib to search path.
-I: Adding /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/parm to search path.
-I: Adding /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/cmd to search path.
-f: Source system.tleap.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./system.tleap
----- Source: /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/cmd/leaprc.RNA.LJbb
----- Source of /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/cmd/leaprc.RNA.LJbb done
Log file: ./leap.log
Loading parameters: /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading library: /gene/home/ldw/miniconda3/envs/amber_test/dat/leap/lib/nucleic12.LJbb-RNA.lib
Loading library: /

In [2]:
import os
import subprocess
import numpy as np
import math
import sys


def hie_pocket_topN(work_pwd, parm_pwd, traj_pwds, out_pwd, out_prefix, frame_per_ns, frame_interval, start_ns, end_ns, pocket_resids, N=1, nofit=False):
    # cluster setting
    if len(pocket_resids) > 0:
        align_resids = ":"
        for start_pos, end_pos in pocket_resids:
            align_resids += f"{start_pos}-{end_pos},"
        align_selection = align_resids + "@C3'"
    else:
        align_selection = "@C3'"
    if nofit:
        fit_parm = "nofit"
    else:
        fit_parm = ""
    # file name
    start_pwd = os.getcwd()
    os.chdir(work_pwd)
    start_frame = int(frame_per_ns * start_ns)
    end_frame = int(frame_per_ns * end_ns)
    trajins = ""
    for traj_pwd in traj_pwds:
        if os.path.exists(traj_pwd):
            trajins += f"trajin {traj_pwd} {start_frame} {end_frame} {frame_interval}\n"
    with open(f"hie_pocket_{start_ns}-{end_ns}ns_N{N}_i{frame_interval}.cpptraj", "w") as ifile:
        ifile.write(f"""
parm {parm_pwd}
{trajins}
autoimage
rmsd {align_selection}
cluster clusters {N} rms {align_selection} {fit_parm} repout {out_pwd}/{out_prefix}_pocket_{start_ns}-{end_ns}ns_hie repfmt pdb summary {out_pwd}/{out_prefix}_pocket_{start_ns}-{end_ns}ns_hie.dat
run
""")
    subprocess.run(f"cpptraj -i hie_pocket_{start_ns}-{end_ns}ns_N{N}_i{frame_interval}.cpptraj > cpptraj.log", shell=True)
    os.chdir(start_pwd)
    return

def find_all_positions(text, pattern):
    positions = []
    start = 0
    while start < len(text):
        idx = text.find(pattern, start)
        if idx == -1:
            break
        positions.append(idx)
        start = idx + 1  # 从下一个位置继续搜索
    return positions

def get_pocket_resid(seq, pocket_seq):
    if pocket_seq == None:
        return []
    pocket_seqs = pocket_seq.strip("...").split("...")
    pocket_resids = []
    for sub_seq in pocket_seqs:
        positions = find_all_positions(seq, sub_seq)
        if len(positions) != 1:
            raise
        pocket_resids.append((positions[0], positions[0]+len(sub_seq)))
    return pocket_resids


traj_ns = 500  # 500 ns
num_of_replica = 3  # 平行轨迹数
box_d = 12  # 水盒子大小
k_m = 0.12  # 0.12 M K+
mg_m = 0.02  # 0.02 M Mg2+
temp = 400  # 400 K
ff = "ljbb"  # RNA force field
wat_ff = "opc3"  # water force field
input_txt = "/gene/home/ldw/Desktop/RNA_ensemble/input/dna_repair.txt"
save_dir = "/gene/home/ldw/Desktop/RNA_ensemble/input/dna_repair"  # 初始pdb结构路径
md_pwd = "/gene/home/ldw/Desktop/RNA_ensemble/md/dna_repair"  # md路径
out_pwd = "/gene/home/ldw/Desktop/RNA_ensemble/analysis/dna_repair.txt"  # 聚类输出路径
frame_per_ns = 10
start_ns = 0
end_ns = traj_ns
N = 10  # 聚类输出结构数目
frame_interval = 2

os.makedirs(out_pwd, exist_ok=True)
rna_infos = {}
with open(input_txt, "r") as ifile:
    for line in ifile:
        words = line.split()
        rna_infos[words[0]] = {"seq": words[1], "rna_ss": words[2]}
        pocket_resids = []
        for pocket_resid in words[3:]:
            pocket_resid = pocket_resid.split("-")
            pocket_resids.append((int(pocket_resid[0]), int(pocket_resid[1])))
        rna_infos[words[0]]["pocket_resids"] = pocket_resids

for rna_name in list(rna_infos.keys())[:]:
    md_name = f"{rna_name}_{ff}_{wat_ff}_{temp}K_{traj_ns}ns_{k_m}k_{mg_m}mg"
    work_pwd = os.path.join(md_pwd, md_name)
    parm_pwd = "system.nosol.top"
    traj_pwds = [f"system.{temp}K.{i}.nc" for i in range(num_of_replica)]
    pocket_resids = rna_infos[rna_name]["pocket_resids"]
    print(rna_name, pocket_resids)
    hie_pocket_topN(work_pwd, parm_pwd, traj_pwds, out_pwd, md_name, frame_per_ns, frame_interval, start_ns, end_ns, pocket_resids, N=N, nofit=False)

ValueError: invalid literal for int() with base 10: '15,'